# Dataset

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('../data/WS22-WS23-augmented-train-gpt.tsv', sep='\t')

In [ ]:
data.columns

Index(['article_id', 'wrong_empathy', 'distress', 'emotion', 'gender',
       'education', 'race', 'age', 'income', 'personality_conscientiousness',
       'personality_openess', 'personality_extraversion',
       'personality_agreeableness', 'personality_stability',
       'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy',
       'iri_empathatic_concern', 'speaker_id', 'demographic_essay', 'article',
       'empathy'],
      dtype='object')

In [ ]:
data.rename(columns={'wrong_empathy': 'crowdsourced_empathy', 'empathy': 'gpt_empathy'}, inplace=True)
data.columns

Index(['article_id', 'crowdsourced_empathy', 'distress', 'emotion', 'gender',
       'education', 'race', 'age', 'income', 'personality_conscientiousness',
       'personality_openess', 'personality_extraversion',
       'personality_agreeableness', 'personality_stability',
       'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy',
       'iri_empathatic_concern', 'speaker_id', 'demographic_essay', 'article',
       'gpt_empathy'],
      dtype='object')

In [ ]:
data.to_csv('../data/v2_v3_train_augmented.tsv', sep='\t', index=False)

In [ ]:
dev_gpt = pd.read_csv('../../LLM-GEm/data/WS23-dev-gpt.tsv', sep='\t')
dev_gpt.columns, dev_gpt.shape

(Index(['conversation_id', 'article_id', 'essay', 'speaker_id', 'gender',
        'education', 'race', 'age', 'income', 'speaker_number', 'split',
        'essay_id', 'demographic', 'demographic_essay', 'article', 'empathy'],
       dtype='object'),
 (208, 16))

In [ ]:
dev_gpt.rename(columns={'empathy': 'gpt_empathy'}, inplace=True)

In [ ]:
dev_original = pd.read_csv('../../LLM-GEm/data/WASSA23/goldstandard_dev.tsv', sep='\t', header=None)
dev_original.columns, dev_original.shape

(Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64'), (208, 12))

In [ ]:
dev_gpt['crowdsourced_empathy'] = dev_original[0].to_list()

In [ ]:
dev_gpt.head(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id,demographic,demographic_essay,article,gpt_empathy,crowdsourced_empathy
0,1,35,How sad is it that this kind of pain and suffe...,68,2,2,1,21,20000,1,dev,0,I am a 21-year-old female of the White race. I...,I am a 21-year-old female of the White race. I...,A month after Hurricane Matthew hit southweste...,6.2,3.833333
1,4,35,The article is kind of tragic and hits close t...,79,1,6,3,33,64000,1,dev,3,I am a 33-year-old male of the Black or Africa...,I am a 33-year-old male of the Black or Africa...,A month after Hurricane Matthew hit southweste...,6.5,3.000000


In [ ]:
dev_gpt.to_csv('../data/v3_dev_gpt.tsv', sep='\t', index=False)